### Notes and Refrences

This document explores a general process for loading, filtering, dimention reduction and clustering using PCA and K-means.

Evalutation is done with distence calculations and silhoutte analysis

See: 

https://towardsdatascience.com/k-means-clustering-algorithm-applications-evaluation-methods-and-drawbacks-aa03e644b48a


Drachen, A., Sifa, R., Bauckhage, C., & Thurau, C. (2012). Guns, swords and data: Clustering of player behavior in computer games in the wild. 2012 IEEE Conference on Computational Intelligence and Games (CIG), 163–170. https://doi.org/10.1109/CIG.2012.6374152


In [0]:

#@markdown ###First Cell 
#@markdown *Please paste this cell into any colab notebook for this project.* <br>
#@markdown ***
#@markdown **Contents**: Contents. <br>
#@markdown ***
#@markdown Major Edit History: 
#@markdown - Author, Date: Created as copy of Old Notebook (`Old Title`).
#@markdown ***
#@markdown <br> 
#@markdown Please change `FIELDDAY_DIR` if it is located differently in your drive. This cell will error if `FIELDDAY_DIR` is incorrect.
# mount drive
from google.colab import drive
drive.mount('/content/drive')

# Change working directory
import os
FIELDDAY_DIR = '/content/drive/My Drive/Field Day' #@param {type:"string"}
JUPYTER_DIR = os.path.join(FIELDDAY_DIR,'Research and Writing Projects/2020 CHI Play - Lakeland Clustering/Jupyter')
os.chdir(JUPYTER_DIR)
print(f'---\nCWD: {os.getcwd()}')

#@markdown Change pandas `max_rows` and `max_columns`
import pandas as pd
pd.options.display.max_columns = 100 #@param {type:"integer"}
pd.options.display.max_rows = 60 #@param {type:"integer"}

#@markdown *Note: There may be other variables to manually change. Look the "Set Variables" section.*

# import utils
import sys
sys.path.append('.')
import utils

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
---
CWD: /content/drive/My Drive/Field Day/Research and Writing Projects/2020 CHI Play - Lakeland Clustering/Jupyter


### Imports

In [0]:
import pandas as pd
from matplotlib import pyplot as plt
from math import ceil
import numpy as np
pd.options.display.max_columns = 1000
from google.colab import files
import urllib.request
from zipfile import ZipFile
from io import BytesIO
import Notebooks.Clustering.cluster_utils as cu
import ipywidgets as widgets
import importlib
from scipy import stats
importlib.reload(cu)

<module 'Notebooks.Clustering.cluster_utils' from '/content/drive/My Drive/Field Day/Research and Writing Projects/2020 CHI Play - Lakeland Clustering/Jupyter/Notebooks/Clustering/cluster_utils.py'>

### Set Vars

In [0]:
options = None#cu.options.lakeland_player_lvl0
print(*str(options).split(','), sep='\n')

None


### Get Data

In [0]:
DecJanLogDF, import_meta = cu.getCrystalDecJanLogDF()
print(*import_meta,sep='\n')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (834,837,839) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


['Import from fData/Raw Log Data/WAVES_20191201_to_20191231_de09c18_proc.zip']
['Import from fData/Raw Log Data/WAVES_20200101_to_20200131_de09c18_proc.zip']


In [0]:
print(DecJanLogDF.shape)
DecJanLogDF.head()

(18284, 1073)


,lvl0_totalSliderMoves,lvl1_totalSliderMoves,lvl2_totalSliderMoves,lvl3_totalSliderMoves,lvl4_totalSliderMoves,lvl5_totalSliderMoves,lvl6_totalSliderMoves,lvl7_totalSliderMoves,lvl8_totalSliderMoves,lvl9_totalSliderMoves,lvl10_totalSliderMoves,lvl11_totalSliderMoves,lvl12_totalSliderMoves,lvl13_totalSliderMoves,lvl14_totalSliderMoves,lvl15_totalSliderMoves,lvl16_totalSliderMoves,lvl17_totalSliderMoves,lvl18_totalSliderMoves,lvl19_totalSliderMoves,lvl20_totalSliderMoves,lvl21_totalSliderMoves,lvl22_totalSliderMoves,lvl23_totalSliderMoves,lvl24_totalSliderMoves,lvl25_totalSliderMoves,lvl26_totalSliderMoves,lvl27_totalSliderMoves,lvl28_totalSliderMoves,lvl29_totalSliderMoves,lvl30_totalSliderMoves,lvl31_totalSliderMoves,lvl32_totalSliderMoves,lvl33_totalSliderMoves,lvl34_totalSliderMoves,lvl0_totalArrowMoves,lvl1_totalArrowMoves,lvl2_totalArrowMoves,lvl3_totalArrowMoves,lvl4_totalArrowMoves,lvl5_totalArrowMoves,lvl6_totalArrowMoves,lvl7_totalArrowMoves,lvl8_totalArrowMoves,lvl9_totalArrowMoves,lvl10_totalArrowMoves,lvl11_totalArrowMoves,lvl12_totalArrowMoves,lvl13_totalArrowMoves,lvl14_totalArrowMoves,lvl15_totalArrowMoves,lvl16_totalArrowMoves,lvl17_totalArrowMoves,lvl18_totalArrowMoves,lvl19_totalArrowMoves,lvl20_totalArrowMoves,lvl21_totalArrowMoves,lvl22_totalArrowMoves,lvl23_totalArrowMoves,lvl24_totalArrowMoves,lvl25_totalArrowMoves,lvl26_totalArrowMoves,lvl27_totalArrowMoves,lvl28_totalArrowMoves,lvl29_totalArrowMoves,lvl30_totalArrowMoves,lvl31_totalArrowMoves,lvl32_totalArrowMoves,lvl33_totalArrowMoves,lvl34_totalArrowMoves,lvl0_totalLevelTime,lvl1_totalLevelTime,lvl2_totalLevelTime,lvl3_totalLevelTime,lvl4_totalLevelTime,lvl5_totalLevelTime,lvl6_totalLevelTime,lvl7_totalLevelTime,lvl8_totalLevelTime,lvl9_totalLevelTime,lvl10_totalLevelTime,lvl11_totalLevelTime,lvl12_totalLevelTime,lvl13_totalLevelTime,lvl14_totalLevelTime,lvl15_totalLevelTime,lvl16_totalLevelTime,lvl17_totalLevelTime,lvl18_totalLevelTime,lvl19_totalLevelTime,lvl20_totalLevelTime,lvl21_totalLevelTime,lvl22_totalLevelTime,lvl23_totalLevelTime,lvl24_totalLevelTime,lvl25_totalLevelTime,lvl26_totalLevelTime,lvl27_totalLevelTime,lvl28_totalLevelTime,lvl29_totalLevelTime,lvl30_totalLevelTime,lvl31_totalLevelTime,lvl32_totalLevelTime,lvl33_totalLevelTime,lvl34_totalLevelTime,lvl0_sliderAvgStdDevs,lvl1_sliderAvgStdDevs,lvl2_sliderAvgStdDevs,lvl3_sliderAvgStdDevs,lvl4_sliderAvgStdDevs,lvl5_sliderAvgStdDevs,lvl6_sliderAvgStdDevs,lvl7_sliderAvgStdDevs,lvl8_sliderAvgStdDevs,lvl9_sliderAvgStdDevs,lvl10_sliderAvgStdDevs,lvl11_sliderAvgStdDevs,lvl12_sliderAvgStdDevs,lvl13_sliderAvgStdDevs,lvl14_sliderAvgStdDevs,lvl15_sliderAvgStdDevs,lvl16_sliderAvgStdDevs,lvl17_sliderAvgStdDevs,lvl18_sliderAvgStdDevs,lvl19_sliderAvgStdDevs,lvl20_sliderAvgStdDevs,lvl21_sliderAvgStdDevs,lvl22_sliderAvgStdDevs,lvl23_sliderAvgStdDevs,lvl24_sliderAvgStdDevs,lvl25_sliderAvgStdDevs,lvl26_sliderAvgStdDevs,lvl27_sliderAvgStdDevs,lvl28_sliderAvgStdDevs,lvl29_sliderAvgStdDevs,lvl30_sliderAvgStdDevs,lvl31_sliderAvgStdDevs,lvl32_sliderAvgStdDevs,lvl33_sliderAvgStdDevs,lvl34_sliderAvgStdDevs,lvl0_totalMoveTypeChanges,lvl1_totalMoveTypeChanges,lvl2_totalMoveTypeChanges,lvl3_totalMoveTypeChanges,lvl4_totalMoveTypeChanges,lvl5_totalMoveTypeChanges,lvl6_totalMoveTypeChanges,lvl7_totalMoveTypeChanges,lvl8_totalMoveTypeChanges,lvl9_totalMoveTypeChanges,lvl10_totalMoveTypeChanges,lvl11_totalMoveTypeChanges,lvl12_totalMoveTypeChanges,lvl13_totalMoveTypeChanges,lvl14_totalMoveTypeChanges,lvl15_totalMoveTypeChanges,lvl16_totalMoveTypeChanges,lvl17_totalMoveTypeChanges,lvl18_totalMoveTypeChanges,lvl19_totalMoveTypeChanges,lvl20_totalMoveTypeChanges,lvl21_totalMoveTypeChanges,lvl22_totalMoveTypeChanges,lvl23_totalMoveTypeChanges,lvl24_totalMoveTypeChanges,lvl25_totalMoveTypeChanges,lvl26_totalMoveTypeChanges,lvl27_totalMoveTypeChanges,lvl28_totalMoveTypeChanges,lvl29_totalMoveTypeChanges,lvl30_totalMoveTypeChanges,lvl31_totalMoveTypeChanges,lvl32_totalMoveTypeChanges,lvl33_totalMoveTypeChanges,lvl34_totalMoveT

### Filter

In [0]:
if options and options.filter_args is not None:
    filtered_df, filter_meta = cu.filter_df(DecJanLogDF, **options.filter_args)
else:
    filtered_df, filter_meta = cu.filter_df(DecJanLogDF, query_list=[], verbose=False, fillna=0)

print(*filter_meta,sep='\n')
filtered_df

UndefinedVariableError: ignored

### Create New Features



In [0]:
if options and options.new_feat_args is not None:
    new_feat_df, new_feat_meta = cu.create_new_base_features(filtered_df, **options.new_feat_args)
else:
    new_feat_df, new_feat_meta = cu.create_new_base_features(filtered_df, avg_tile_hover_lvl_range=range(0,2))
print(*new_feat_meta,sep='\n')
new_feat_df

### Optional: Select Level Features to Aggregate
Ignore the 'sess' and 'other' columns.<br>
Or choose to use hardcoded options

In [0]:
if not options: #options.lvlfeats is not None and options.lvlrange is not None:
    level_features_widget = cu.get_feat_selection(new_feat_df)
    display(level_features_widget)

### Select final features

In [0]:
if options:# and options.lvlfeats is not None and options.lvlrange is not None: 
    aggregate_df, aggregate_meta = cu.describe_lvl_feats(new_feat_df, options.lvlfeats, options.lvlrange)
else:
    lvl_feats, lvl_range = cu.get_level_feats_and_range(level_features_widget)
    aggregate_df, aggregate_meta = cu.describe_lvl_feats(new_feat_df, lvl_feats, lvl_range)
# if not options:# and options.finalfeats is not None:
    final_features_widget = cu.get_feat_selection(aggregate_df)
    display(final_features_widget)
print(*aggregate_meta,sep='\n')

### Output final columns

In [0]:
if options: #None not in [options, options.finalfeats]:
    reduced_df, reduced_meta = cu.reduce_feats(aggregate_df, options.finalfeats)
else:
    final_feats = cu.get_selected_feature_list(final_features_widget)
    reduced_df, reduced_meta = cu.reduce_feats(aggregate_df, final_feats)
print(*reduced_meta, sep='\n')
reduced_df

In [0]:
reduced_df.describe()

In [0]:
if not options:
    final_df, outlier_meta = cu.reduce_outliers(reduced_df, 3)
else:
    final_df, outlier_meta = cu.reduce_outliers(reduced_df, options.zthresh)
final_meta = import_meta + filter_meta + new_feat_meta + aggregate_meta + reduced_meta + outlier_meta
print(*final_meta, sep='\n')

In [0]:
cu.print_options(final_meta)